In [9]:
import requests
import pandas as pd
import numpy as np
import json
import os

class ConnectAPI:

    def __init__(self):
        self.api_token = 'test85g57'
        self.season_id = '2012'
        self.matches= "https://api.football-data-api.com/league-matches?key={api_token}&season_id={season_id}"
        self.team = "https://api.football-data-api.com/league-teams?key={api_token}&season_id={season_id}&include=stats"
        self.current_path = os.getcwd().partition('notebooks')[0]
        
    def get_json(self, url:str, **kwargs) -> json: 
        url = url.format(api_token = self.api_token, **kwargs)
        self.response = requests.get(url)
        print(self.response.status_code,'status_code')
        print(self.response.request,'request')
        jObject = self.response.json()
        return jObject
    
class MyFunctions:

    def open_json(project_path:str, relative_path:str, file_name:str):
        jsobject = open(project_path + relative_path + "\\" + file_name, 'r')
        json_file = json.load(jsobject)
        return json_file
    
class TableCreating:
    
    def create_table(df:pd.DataFrame, home_away:str) -> pd.DataFrame:
        if home_away == 'home':
            goals_scored = ('homeGoalCount', 'sum')
            goals_concened= ('awayGoalCount', 'sum')
        elif home_away == 'away':
            goals_scored = ('awayGoalCount', 'sum')
            goals_concened= ('homeGoalCount', 'sum')
            
        df = df.groupby([f'{home_away}_team', f'{home_away}ID'] ).agg(
        matches_played = (f'{home_away}_team', 'count'),
        goals_scored = goals_scored,
        goals_concened= goals_concened,    
        ).reset_index()
        
        return df
    
    def create_result_table(df, home_away:str) -> pd.DataFrame:
        
        df = df[[f'{home_away}ID',f'{home_away}_team', 'result']]
        pivot_df = pd.pivot_table(df, index=[f'{home_away}ID', f'{home_away}_team'], columns='result', aggfunc=len, fill_value=0).reset_index() #kaŽkodėl čia len sumuoja :\
        
        if home_away == 'home': df = pivot_df[['homeID','home_team', 'home_win', 'draw', 'away_win']].rename(
            columns={
                'homeID':'teamID',
                'home_team':'team',
                'home_win':'win',
                "away_win": "losse"})
        elif home_away == 'away': df = pivot_df[['awayID','away_team', 'away_win', 'draw', 'home_win']].rename(
            columns={
                'awayID':'teamID',
                'away_team':'team',
                'away_win':'win',
                "home_win": "losse"})
            
        return df
        
class Tables(ConnectAPI):
    
    def __init__(self):
        super().__init__()
        self.create_teams_table()

    def create_teams_table(self) -> pd.DataFrame:
        jsondb_teams = self.get_json(url = self.team, season_id = self.season_id)
        global TEAMS_df
        TEAMS_df = pd.DataFrame(jsondb_teams['data'])
        
        
    def create_my_table(self, json_object:json, data:str = 'data', columns:list = []) -> pd.DataFrame:
        """data default: 'data' 
           column default: empty list"""
        df = pd.DataFrame(json_object[data])
        if columns == []: 
            return df
        else: 
            return df[columns]
        
    def create_general_table(self) -> pd.DataFrame:
        
        json_columns = MyFunctions.open_json(project_path=dt.current_path, relative_path="json", file_name='tables_headers.json')
        columns = json_columns['tables']['general_table']['columns']
        jsondb_matches = dt.get_json(url = dt.matches, season_id = dt.season_id)
        df = self.create_my_table(json_object=jsondb_matches, columns=columns)
        
        matches = pd.merge(df, TEAMS_df[['id','name']], how = 'left', left_on='homeID', right_on='id') 
        matches2 = pd.merge(matches, TEAMS_df[['id','name']], how = 'left', left_on='awayID', right_on='id' ) 
        df = matches2.drop(columns=['id', 'id_y']).rename(columns={"id_x":"id", 'name_x':'home_team','name_y':'away_team'}) 
    
        df = df[['id', 'home_team', 'away_team', 'season', 'status', 'roundID','game_week', 'homeGoals', 
               'awayGoals', 'homeGoalCount', 'awayGoalCount', 'totalGoalCount', 'homeID', 'awayID']]
        
        df['result'] = np.where(df['homeGoals'] > df['awayGoals'], 'home_win', 
                        np.where(df['homeGoals'] < df['awayGoals'], 'away_win', 'draw'))
        
        return df
        
        
    def create_home_table(self, df:pd.DataFrame) -> pd.DataFrame:
        df = TableCreating.create_table(df, 'home')
        
        
        
        return df
    
        
        
        
        
class Calculate:
    
    def __init__(self):
        pass

    
if __name__ == "__main__":
    
    dt = Tables()
    matches = dt.create_general_table()
    home = dt.create_home_table(matches)
    df = TableCreating.create_result_table(matches, 'home')

    


200 status_code
<PreparedRequest [GET]> request


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\football_data_analysis\\football-matches-analysis\\json\\tables_headers.json'

In [1]:
import requests
import logging
import sys

sys.path.append("..\\config\\")
import secret
import config

sys.path.append("..\\scripts\\modules\\")
import data_pocessing as dtp

config.log


def connect_to_footystats(url: str, **kwargs):
    url = url.format(api_token=secret.API_TOKEN, season_id=secret.SEASON_ID, **kwargs)
    response = requests.get(url)
    logging.info(response.status_code)
    logging.info(response.request)
    return response


def get_json_API(response: requests):
    jsonObj = response.json()
    return jsonObj


if __name__ == "__main__":
    response = connect_to_footystats(url=config.TEAMS_URL)
    js = get_json_API(response)
    df = dtp.create_main_table(js)
    
    
    
    


INFO:root:200
INFO:root:<PreparedRequest [GET]>


In [2]:
df

,id,name,cleanName,english_name,shortHand,country,continent,founded,image,flag_element,...,url,table_position,performance_rank,risk,season_format,competition_id,full_name,alt_names,official_sites,stats
0,59,Arsenal FC,Arsenal,Arsenal FC,arsenal-fc,England,None,1886,https://cdn.footystats.org/img/teams/england-a...,None,...,https://footystats.org/clubs/arsenal-fc-59,8,8,63,Domestic League,2012,Arsenal FC,[The Gunners],[https://www.arsenal.com/],"{'previous_seasons': [], 'suspended_matches': ..."
1,92,Tottenham Hotspur FC,Tottenham Hotspur,Tottenham Hotspur FC,tottenham-hotspur-fc,England,None,1882,https://cdn.footystats.org/img/teams/england-t...,None,...,https://footystats.org/clubs/tottenham-hotspur...,6,6,68,Domestic League,2012,Tottenham Hotspur FC,[Spurs / The Lilywhites],[https://www.tottenhamhotspur.com/],"{'previous_seasons': [], 'suspended_matches': ..."
2,93,Manchester City FC,Manchester City,Manchester City FC,manchester-city-fc,England,None,1880,https://cdn.footystats.org/img/teams/england-m...,None,...,https://footystats.org/clubs/manchester-city-f...,2,2,105,Domestic League,2012,Manchester City FC,[Man City],[https://www.mancity.com/],"{'previous_seasons': [], 'suspended_matches': ..."
3,108,Leicester City FC,Leicester City,Leicester City FC,leicester-city-fc,England,None,1884,https://cdn.footystats.org/img/teams/england-l...,None,...,https://footystats.org/clubs/leicester-city-fc...,5,5,82,Domestic League,2012,Leicester City FC,[Leicester],[https://www.lcfc.com/],"{'previous_seasons': [], 'suspended_matches': ..."
4,143,Crystal Palace FC,Crystal Palace,Crystal Palace FC,crystal-palace-fc,England,None,1905,https://cdn.footystats.org/img/teams/england-c...,None,...,https://footystats.org/clubs/crystal-palace-fc...,14,14,39,Domestic League,2012,Crystal Palace FC,[Eagles / Glaziers / Palace],[https://www.cpfc.co.uk/],"{'previous_seasons': [], 'suspended_matches': ..."
5,144,Everton FC,Everton,Everton FC,everton-fc,England,None,1878,https://cdn.footystats.org/img/teams/england-e...,None,...,https://footystats.org/clubs/everton-fc-144,12,12,61,Domestic League,2012,Everton FC,[The Toffees / The Blues],[https://www.evertonfc.com/],"{'previous_seasons': [], 'suspended_matches': ..."
6,145,Burnley FC,Burnley,Burnley FC,burnley-fc,England,None,1882,https://cdn.footystats.org/img/teams/england-b...,None,...,https://footystats.org/clubs/burnley-fc-145,10,10,55,Domestic League,2012,Burnley FC,[The Clarets],[https://www.burnleyfootballclub.com/],"{'previous_seasons': [], 'suspended_matches': ..."
7,146,Southampton FC,Southampton,Southampton FC,southampton-fc,England,None,1885,https://cdn.footystats.org/img/teams/england-s...,None,...,https://footystats.org/clubs/southampton-fc-146,11,11,84,Domestic League,2012,Southampton FC,[Saints],[https://www.southamptonfc.com/],"{'previous_seasons': [], 'suspended_matches': ..."
8,148,AFC Bournemouth,AFC Bournemouth,AFC Bournemouth,afc-bournemouth,England,None,1899,https://cdn.footystats.org/img/teams/england-a...,None,...,https://footystats.org/clubs/afc-bournemouth-148,18,18,87,Domestic League,2012,AFC Bournemouth,[The Cherries / Boscombe],[https://www.afcb.co.uk/],"{'previous_seasons': [], 'suspended_matches': ..."
9,149,Manchester United FC,Manchester United,Manchester United FC,manchester-united-fc,England,None,1878,https://cdn.footystats.org/img/teams/england-m...,None,...,https://footystats.org/clubs/manchester-united...,3,3,74,Domestic League,2012,Manchester United FC,[Man Utd],[https://www.manutd.com/],"{'previous_seasons': [], 'suspended_matches': ..."
